In [ ]:
import mqr

In [ ]:
import numpy as np
import pandas as pd

---
# Tools for specifying levels in experiments

In [ ]:
np.repeat(1, 3), np.tile(1, 3)

In [ ]:
np.repeat([1, 2], 3)

In [ ]:
np.tile([1, 2], 3)

In [ ]:
# Interleaving two levels with increasing group size
# NB: `np.repeat(a, 1) == a` and `np.tile(a, 1) == a` are both `True`.
#     There is no need to use them in practise; they are shown here
#     just to illustrate the pattern.
levels = [1, 2]
display(np.repeat(np.tile(levels, 1), 8))
display(np.repeat(np.tile(levels, 2), 4))
display(np.repeat(np.tile(levels, 4), 2))
display(np.repeat(np.tile(levels, 8), 1))

---
# Creating a `DataFrame` manually

In [ ]:
# ... from an array with manual column name

values = np.array([
    19.8, 10.1, 14.9,  7.5, 15.4, 15.4,
    15.4, 18.5,  7.9, 12.7, 11.9, 11.4,
    11.4, 14.1, 17.6, 16.7, 15.8,
    19.5,  8.8, 13.6, 11.9, 11.4,
])
index = pd.RangeIndex(stop=len(values), name='part')
columns = ['failure load']

load_df = pd.DataFrame(
    values,
    index=index,
    columns=columns
)
load_df

In [ ]:
# ... from a dictionary

primer = np.repeat([1, 2, 3], 6) # 3*6==18
technique = np.tile(np.repeat(['dipping', 'spraying'], 3), 3) # 2*3*3==18
replicate = np.tile([1, 2, 3], 6) # 3*6==18
adhesion_force = np.array([
    4.0, 4.5, 4.3, 4.5, 4.9, 5.6,
    5.6, 4.9, 5.4, 5.8, 6.1, 6.3,
    3.8, 3.7, 4.0, 5.5, 5.0, 5.0])
data = {
    'primer': primer,
    'technique': technique,
    'replicate': replicate,
    'adhesion_force': adhesion_force}

index = pd.RangeIndex(stop=len(adhesion_force))

sample_df = pd.DataFrame(
    data=data,
    index=index)
sample_df

---
# Saving a `DataFrame` to CSV

In [ ]:
# Set `index=False` to omit the auto-generated numerical index
sample_df.to_csv('sample_df.csv', index=False)

---
# Loading a `DataFrame` from CSV

In [ ]:
# The index is auto-generated again when the file is loaded
pd.read_csv('sample_df.csv')

In [ ]:
# Use a named column as the index
pd.read_csv(mqr.sample_data('study-random-5x5.csv'), index_col='run')

---
# Sample datasets

In [ ]:
from importlib import resources
[f.name for f in resources.files('mqr.data').iterdir()]

---
# Arranging multiple `DataFrame`s in a single output

Jupyter normally shows a single output per cell. This tool arranges `DataFrame`s using HTML `div` elements and flex layouts.

It calls `_repr_html_` on arguments, so works with any object that implements that method.

In [ ]:
mqr.nbtools.vstack(
    pd.DataFrame(index=['data'], columns=range(30), data=np.arange(30)[None, :]+100),
    mqr.nbtools.hstack(load_df, sample_df))